In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stat
import statsmodels.api as sm
import statsmodels.formula.api as smf#, ols
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import ttest_ind, f_oneway
import scipy.stats as stats
import sys
from pathlib import Path
# # Add src directory to the Python path
sys.path.append(str(Path.cwd().parent))
from src.data_analysis.df_dataattribute_analysis import get_data_attributes
from src.data_sourcing.import_export_data import get_data, save_data
from src.data_analysis.df_ttest_anova_analysis import run_one_sample_ttest, run_univariate_tests
from src.data_analysis.df_multicollinearity_analysis import compute_vif
from src.data_analysis.df_correlation_analysis import compute_correlations

In [13]:
df = get_data('featured_engineered_aggregations.csv', 'processed_data')
df.head(5)

,ATL_OR_DR,CAMPAIGN_TYPE,CHANNEL,COST,FREE_TRIALS,HOLIDAY_FLAG,SEASON,MONTH,ATL_OR_DR_CAMPAIGN_TYPE,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL,...,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,AVG_COST_PER_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,MEDIAN_COST_PER_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,AVG_COST_BY_CAMPAIGN_TYPE_MONTH,MEDIAN_COST_BY_CAMPAIGN_TYPE_MONTH,COST_OVER_AVG_COST_BY_CAMPAIGN_TYPE_MONTH,NORMALIZE_FREE_TRIAL_BY_MONTH,NORMALIZE_COST_PER_FREE_TRIALS_BY_MONTH,NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH
0,DR - Direct Response,Title,paid social,7784.31,86401.15,False,Summer,June,DR - Direct Response_Title,DR - Direct Response_Title_paid social,...,2.161878,0.942693,0.129226,0.095572,12783.161145,3432.160,0.608950,2.015642,0.086220,0.701295
1,DR - Direct Response,Title,app,2474.31,2956.74,False,Summer,August,DR - Direct Response_Title,DR - Direct Response_Title_app,...,0.070544,0.853002,0.107078,0.981049,14949.239779,3239.655,0.165514,0.067269,1.023058,0.013036
2,ATL - Above The Line,Title,paid social,10222.82,12513.08,False,Summer,July,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,...,0.263725,0.850812,0.101840,0.960225,13099.998683,2802.460,0.780368,0.272262,0.984393,0.226908
3,DR - Direct Response,Title,app,49631.87,17207.04,False,Winter,December,DR - Direct Response_Title,DR - Direct Response_Title_app,...,0.367143,0.871586,0.109911,3.309360,14484.916524,3133.110,3.426452,0.362288,2.645155,0.979314
4,ATL - Above The Line,Title,paid social,2081.06,21758.33,False,Summer,August,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,...,0.519126,0.853002,0.107078,0.112127,14949.239779,3239.655,0.139208,0.495028,0.116928,0.080683


### Multi-collinearity Testing Analysis

In [14]:
# df.columns

# X_df= df.drop(columns=['FREE_TRIALS', 'ATL_OR_DR_CAMPAIGN_TYPE', 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL', 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH',
#                          'ATL_OR_DR_CAMPAIGN_TYPE_MONTH', 'ATL_OR_DR_CHANNEL_MONTH', 'CAMPAIGN_TYPE_CHANNEL_MONTH', 'CAMPAIGN_TYPE_MONTH', 'CHANNEL_MONTH',
#                            'LOG_FREE_TRIALS', 'ATL_OR_DR_MONTH'], axis=1)
# X_df.columns
#
hashmap = get_data_attributes(df)
formula = hashmap['numerical']
formula.remove('FREE_TRIALS')
formula.remove('LOG_FREE_TRIALS')
formula.remove('LOG_COST')
formula.extend(['CHANNEL', 'HOLIDAY_FLAG', 'SEASON']) #'CHANNEL'
print("formula:", formula)
temp_df = df[formula]
vif_result = compute_vif(
    data=temp_df,
)
vif_result
# '''
# Try an aggregation calculation by Channel and month? So 2 group by's?
# '''
vif_result[(vif_result['VIF'] > 5) & (vif_result['VIF'] != float('inf'))]

formula: ['COST', 'COST_PER_FREE_TRIALS', 'COST_BY_FREE_TRIALS', 'LOG_COST_PER_FREE_TRIALS', 'LOG_COST_BY_FREE_TRIALS', 'AVG_COST_MONTH', 'MEDIAN_COST_MONTH', 'AVG_COST_CAMPAIGN_TYPE_CHANNEL', 'MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL', 'AVG_FREE_TRIALS_MONTH', 'MEDIAN_FREE_TRIALS_MONTH', 'AVG_FREE_TRIALS_BY_CHANNEL_MONTH', 'MEDIAN_FREE_TRIALS_BY_CHANNEL_MONTH', 'FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CHANNEL_MONTH', 'AVG_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH', 'MEDIAN_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH', 'COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH', 'AVG_COST_BY_CHANNEL_MONTH', 'MEDIAN_COST_BY_CHANNEL_MONTH', 'COST_OVER_AVG_COST_BY_CHANNEL_MONTH', 'AVG_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'MEDIAN_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'AVG_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'MEDIAN_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'AVG_COST_BY_ATL_OR_D

,Feature,VIF
0,NORMALIZE_FREE_TRIAL_BY_MONTH,832.990394
1,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_ATL_OR_DR_...,656.220944
2,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CAMPAIGN_T...,315.264234
3,NORMALIZE_COST_PER_FREE_TRIALS_BY_MONTH,142.127427
4,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,93.848562
5,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,87.088133
6,COST_OVER_AVG_COST_BY_CAMPAIGN_TYPE_MONTH,85.721714
7,COST_OVER_AVG_COST_BY_ATL_OR_DR_MONTH,78.977738
8,COST,50.035209
9,NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH,46.140415


### One Sample T-Test Aanalysis

In [15]:
categorical_list = ['ATL_OR_DR','HOLIDAY_FLAG']
quantitative = ["FREE_TRIALS", "LOG_FREE_TRIALS"]
avg_free_trials = round(df['FREE_TRIALS'].mean(),2)
avg_log_free_trials = round(df['LOG_FREE_TRIALS'].mean(),2)
# 2) Using a dictionary for different hypothesized means per column:
results_2 = run_one_sample_ttest(
    df=df,
    categorical_cols=categorical_list,
    quantitative_cols=quantitative,
    popmean={"FREE_TRIALS": avg_free_trials, "LOG_FREE_TRIALS": avg_log_free_trials},  # different popmeans
    stat_type="mean",
    alpha=0.05,  # more stringent significance level
    sort_results=True
)
results_2

,Categorical Variable,Category,Quantitative Variable,Sample Size,Sample Mean,T-statistic,p-value,Decision,Error
0,HOLIDAY_FLAG,True,LOG_FREE_TRIALS,534,10.201621,-0.637220,0.524255,Fail to Reject H0,None
1,ATL_OR_DR,ATL - Above The Line,LOG_FREE_TRIALS,8601,10.228078,-0.178557,0.858290,Fail to Reject H0,None
2,ATL_OR_DR,DR - Direct Response,LOG_FREE_TRIALS,5507,10.227902,-0.155741,0.876243,Fail to Reject H0,None
3,ATL_OR_DR,DR - Direct Response,FREE_TRIALS,5507,45547.216388,0.112888,0.910123,Fail to Reject H0,None
4,HOLIDAY_FLAG,False,LOG_FREE_TRIALS,13574,10.229048,-0.111226,0.911438,Fail to Reject H0,None
5,ATL_OR_DR,ATL - Above The Line,FREE_TRIALS,8601,45401.434358,-0.085614,0.931775,Fail to Reject H0,None
6,HOLIDAY_FLAG,True,FREE_TRIALS,534,45484.437959,0.009366,0.992530,Fail to Reject H0,None
7,HOLIDAY_FLAG,False,FREE_TRIALS,13574,45457.313076,-0.001986,0.998415,Fail to Reject H0,None


### ANOVA Testing Analysis

In [16]:
# https://towardsdatascience.com/anova-test-with-python-cfbf4013328b
hashmap = get_data_attributes(df)
categorical = hashmap['categorical']
categorical.remove('ATL_OR_DR')
categorical.remove('HOLIDAY_FLAG')
categorical

['CAMPAIGN_TYPE',
 'CHANNEL',
 'SEASON',
 'MONTH',
 'ATL_OR_DR_CAMPAIGN_TYPE',
 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL',
 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH',
 'ATL_OR_DR_CAMPAIGN_TYPE_MONTH',
 'ATL_OR_DR_CHANNEL',
 'ATL_OR_DR_CHANNEL_MONTH',
 'ATL_OR_DR_MONTH',
 'CAMPAIGN_TYPE_CHANNEL',
 'CAMPAIGN_TYPE_CHANNEL_MONTH',
 'CAMPAIGN_TYPE_MONTH',
 'CHANNEL_MONTH']

In [17]:
quantitative = ["FREE_TRIALS", "LOG_FREE_TRIALS"]

anova_results_df = run_univariate_tests(
    df=df,
    categorical_cols=categorical,
    quantitative_cols=quantitative,
    test_type="anova"
)
anova_results_df[anova_results_df['p-value'] <= 0.50]

,Categorical Variable,Quantitative Variable,F-statistic,p-value,Error
0,MONTH,FREE_TRIALS,1.745376,0.057730,None
1,ATL_OR_DR_MONTH,FREE_TRIALS,1.437204,0.080345,None
2,CAMPAIGN_TYPE_MONTH,FREE_TRIALS,1.346628,0.104410,None
3,CHANNEL_MONTH,FREE_TRIALS,1.096844,0.170871,None
4,MONTH,LOG_FREE_TRIALS,1.335306,0.197357,None
5,CAMPAIGN_TYPE_CHANNEL_MONTH,FREE_TRIALS,1.061845,0.206772,None
6,ATL_OR_DR_CAMPAIGN_TYPE_MONTH,FREE_TRIALS,1.112372,0.268880,None
7,ATL_OR_DR_CHANNEL_MONTH,FREE_TRIALS,1.050398,0.285439,None
8,ATL_OR_DR_MONTH,LOG_FREE_TRIALS,1.085444,0.352263,None
9,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH,FREE_TRIALS,1.023266,0.361860,None


### Kruskal Wallis Testing Analysis

In [18]:
kruskal_results_df = run_univariate_tests(
    df=df,
    categorical_cols=categorical,
    quantitative_cols=quantitative,
    test_type="kruskal"
)
kruskal_results_df[kruskal_results_df['p-value'] <= 0.60]

,Categorical Variable,Quantitative Variable,H-statistic,p-value,Error
0,MONTH,FREE_TRIALS,15.421565,0.163999,None
1,MONTH,LOG_FREE_TRIALS,15.421565,0.163999,None
2,ATL_OR_DR_MONTH,LOG_FREE_TRIALS,26.406948,0.282034,None
3,ATL_OR_DR_MONTH,FREE_TRIALS,26.406948,0.282034,None
4,SEASON,FREE_TRIALS,2.744503,0.432717,None
5,SEASON,LOG_FREE_TRIALS,2.744503,0.432717,None
6,CHANNEL_MONTH,LOG_FREE_TRIALS,193.675803,0.493063,None
7,CHANNEL_MONTH,FREE_TRIALS,193.675803,0.493063,None
8,ATL_OR_DR_CHANNEL_MONTH,LOG_FREE_TRIALS,233.572700,0.586903,None
9,ATL_OR_DR_CHANNEL_MONTH,FREE_TRIALS,233.572700,0.586903,None


### Pearson Correlation Testing Analysis

In [19]:
# 1) Pearson correlation (numeric only - only linear relationships)
pearson_results = compute_correlations(
    df=df,
    include_categorical=False,
    cat_cols=None  # Do not encode categorical variables
)
pearson_results[
    (pearson_results['p-value'] < 0.05) &
    (pearson_results['Variable 1'] == 'FREE_TRIALS')
]

,Variable 1,Variable 2,Correlation,p-value,Method,Error
100,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CHANNEL_MONTH,0.972843,0.000000e+00,pearson,None
113,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CAMPAIGN_T...,0.995560,0.000000e+00,pearson,None
127,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_ATL_OR_DR_...,0.996181,0.000000e+00,pearson,None
152,FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,0.347763,0.000000e+00,pearson,None
153,FREE_TRIALS,LOG_FREE_TRIALS,0.747446,0.000000e+00,pearson,None
158,FREE_TRIALS,NORMALIZE_FREE_TRIAL_BY_MONTH,0.997894,0.000000e+00,pearson,None
192,FREE_TRIALS,NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH,0.230571,1.356582e-169,pearson,None
201,FREE_TRIALS,COST_BY_FREE_TRIALS,0.219831,5.792956e-154,pearson,None
208,FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,-0.215401,9.361971e-148,pearson,None
291,FREE_TRIALS,AVG_FREE_TRIALS_BY_CHANNEL_MONTH,0.122735,1.752471e-48,pearson,None


### Spearman Correlation Testing Analysis

In [20]:
# 2) Spearman correlation (with categorical one-hot encoding)
spearman_results = compute_correlations(
    df=df,
    include_categorical=True,  # Encode categorical variables
    drop_first=True,
    cat_cols=categorical
)
spearman_results[(spearman_results['Variable 1'] == 'FREE_TRIALS') & (spearman_results['p-value'] < 0.05)]

#spearman_results[((spearman_results['Variable 1'] == 'FREE_TRIALS') | (spearman_results['Variable 1'] == 'LOG_FREE_TRIALS')) & (spearman_results['p-value'] < 0.05)]

,Variable 1,Variable 2,Correlation,p-value,Method,Error
130,FREE_TRIALS,COST_PER_FREE_TRIALS,-0.460507,0.000000e+00,spearman,None
131,FREE_TRIALS,COST_BY_FREE_TRIALS,0.470630,0.000000e+00,spearman,None
133,FREE_TRIALS,LOG_FREE_TRIALS,1.000000,0.000000e+00,spearman,None
134,FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,-0.460507,0.000000e+00,spearman,None
135,FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,0.470630,0.000000e+00,spearman,None
141,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CHANNEL_MONTH,0.986150,0.000000e+00,spearman,None
144,FREE_TRIALS,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,-0.449598,0.000000e+00,spearman,None
145,FREE_TRIALS,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,-0.459360,0.000000e+00,spearman,None
148,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CAMPAIGN_T...,0.997544,0.000000e+00,spearman,None
152,FREE_TRIALS,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,-0.458958,0.000000e+00,spearman,None


### Random Testing Analysis

In [21]:
formula

['COST',
 'COST_PER_FREE_TRIALS',
 'COST_BY_FREE_TRIALS',
 'LOG_COST_PER_FREE_TRIALS',
 'LOG_COST_BY_FREE_TRIALS',
 'AVG_COST_MONTH',
 'MEDIAN_COST_MONTH',
 'AVG_COST_CAMPAIGN_TYPE_CHANNEL',
 'MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL',
 'AVG_FREE_TRIALS_MONTH',
 'MEDIAN_FREE_TRIALS_MONTH',
 'AVG_FREE_TRIALS_BY_CHANNEL_MONTH',
 'MEDIAN_FREE_TRIALS_BY_CHANNEL_MONTH',
 'FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CHANNEL_MONTH',
 'AVG_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH',
 'MEDIAN_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH',
 'COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH',
 'AVG_COST_BY_CHANNEL_MONTH',
 'MEDIAN_COST_BY_CHANNEL_MONTH',
 'COST_OVER_AVG_COST_BY_CHANNEL_MONTH',
 'AVG_FREE_TRIALS_BY_ATL_OR_DR_MONTH',
 'MEDIAN_FREE_TRIALS_BY_ATL_OR_DR_MONTH',
 'FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_ATL_OR_DR_MONTH',
 'AVG_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH',
 'MEDIAN_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH',
 'COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH',
 'AVG

In [33]:
def prepare_and_run_ols(categorical_vars_df, quantitative_vars_df, target_variable, drop_first=True):
    # Create copy to avoid modifying original data
    categorical_vars = categorical_vars_df.copy()
    quantitative_vars = quantitative_vars_df.copy()

    # Initialize encoder
    encoder = OneHotEncoder(drop='first' if drop_first else None, sparse=False)

    # Fit and transform categorical variables
    categorical_encoded = encoder.fit_transform(categorical_vars)

    # Get feature names for encoded variables
    encoded_feature_names = []
    for i, feature in enumerate(categorical_vars.columns):
        cats = encoder.categories_[i]
        if drop_first:
            cats = cats[1:]  # Skip first category if dropping
        encoded_feature_names.extend([f"{feature}_{cat}" for cat in cats])

    # Create DataFrame with encoded categorical variables
    categorical_df = pd.DataFrame(
        categorical_encoded,
        columns=encoded_feature_names,
        index=categorical_vars.index
    )

    # Combine categorical and quantitative variables
    X = pd.concat([categorical_df, quantitative_vars], axis=1)

    # Add constant for intercept
    X = sm.add_constant(X)

    # Fit OLS model
    model = sm.OLS(target_variable, X)
    results = model.fit()

    # Calculate VIF scores
    vif_data = calculate_vif(X)

    return results, vif_data

def calculate_vif(X):
    vif_df = pd.DataFrame()
    vif_df["Variable"] = X.columns
    vif_df["VIF"] = [variance_inflation_factor(X.values, i)
                     for i in range(X.shape[1])]
    return vif_df.sort_values('VIF', ascending=False)

# Example usage:
"""
# Your data should already be in DataFrame format:
categorical_vars_df = df[['ATL_OR_DR', 'CAMPAIGN_TYPE', 'CHANNEL', 'HOLIDAY_FLAG', 'MONTH']]
quantitative_vars_df = df[['COST', 'COST_PER_FREE_TRIALS', 'COST_BY_FREE_TRIALS']]
target = df['FREE_TRIALS']

# Run the model
model_results, vif_scores = prepare_and_run_ols(
    categorical_vars_df=categorical_vars_df,
    quantitative_vars_df=quantitative_vars_df,
    target_variable=target
)

# Print results
print(model_results.summary())
print("\nVariance Inflation Factors:")
print(vif_scores)
"""

'\n# Your data should already be in DataFrame format:\ncategorical_vars_df = df[[\'ATL_OR_DR\', \'CAMPAIGN_TYPE\', \'CHANNEL\', \'HOLIDAY_FLAG\', \'MONTH\']]\nquantitative_vars_df = df[[\'COST\', \'COST_PER_FREE_TRIALS\', \'COST_BY_FREE_TRIALS\']]\ntarget = df[\'FREE_TRIALS\']\n\n# Run the model\nmodel_results, vif_scores = prepare_and_run_ols(\n    categorical_vars_df=categorical_vars_df,\n    quantitative_vars_df=quantitative_vars_df,\n    target_variable=target\n)\n\n# Print results\nprint(model_results.summary())\nprint("\nVariance Inflation Factors:")\nprint(vif_scores)\n'

In [35]:
categorical_vars = get_data_attributes(df)['categorical']
quantitative_vars = get_data_attributes(df)['numerical']
quantitative_vars.remove('FREE_TRIALS')
quantitative_vars.remove('LOG_FREE_TRIALS')
categorical_vars = df[categorical_vars]
quantitative_vars = df[quantitative_vars]
target = df['FREE_TRIALS']

# Run the model
model_results, vif_scores = prepare_and_run_ols(
    categorical_vars_df=categorical_vars,
    quantitative_vars_df=quantitative_vars,
    target_variable=target
)

# Print summary
print(model_results.summary())

# Check VIF scores
print("\nVariance Inflation Factors:")
print(vif_scores)

# Access specific statistics
print("\nR-squared:", model_results.rsquared)
print("Adjusted R-squared:", model_results.rsquared_adj)
print("F-statistic p-value:", model_results.f_pvalue)

c:\Users\cheng\Workspace\Paramount\.venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\cheng\Workspace\Paramount\.venv\lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


KeyboardInterrupt: 

In [29]:
categorical_vars

['ATL_OR_DR',
 'CAMPAIGN_TYPE',
 'CHANNEL',
 'HOLIDAY_FLAG',
 'SEASON',
 'MONTH',
 'ATL_OR_DR_CAMPAIGN_TYPE',
 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL',
 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH',
 'ATL_OR_DR_CAMPAIGN_TYPE_MONTH',
 'ATL_OR_DR_CHANNEL',
 'ATL_OR_DR_CHANNEL_MONTH',
 'ATL_OR_DR_MONTH',
 'CAMPAIGN_TYPE_CHANNEL',
 'CAMPAIGN_TYPE_CHANNEL_MONTH',
 'CAMPAIGN_TYPE_MONTH',
 'CHANNEL_MONTH']

In [27]:
np.array(categorical_vars).reshape(-1, 1)

array([['ATL_OR_DR'],
       ['CAMPAIGN_TYPE'],
       ['CHANNEL'],
       ['HOLIDAY_FLAG'],
       ['SEASON'],
       ['MONTH'],
       ['ATL_OR_DR_CAMPAIGN_TYPE'],
       ['ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL'],
       ['ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH'],
       ['ATL_OR_DR_CAMPAIGN_TYPE_MONTH'],
       ['ATL_OR_DR_CHANNEL'],
       ['ATL_OR_DR_CHANNEL_MONTH'],
       ['ATL_OR_DR_MONTH'],
       ['CAMPAIGN_TYPE_CHANNEL'],
       ['CAMPAIGN_TYPE_CHANNEL_MONTH'],
       ['CAMPAIGN_TYPE_MONTH'],
       ['CHANNEL_MONTH']], dtype='<U37')

In [10]:
formula = """
FREE_TRIALS ~ HOLIDAY_FLAG
+ ATL_OR_DR + CAMPAIGN_TYPE + CHANNEL + MONTH
+ ATL_OR_DR_CAMPAIGN_TYPE + ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL
+ ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH + ATL_OR_DR_CAMPAIGN_TYPE_MONTH
+ ATL_OR_DR_CHANNEL + ATL_OR_DR_CHANNEL_MONTH + ATL_OR_DR_MONTH
+ CAMPAIGN_TYPE_CHANNEL + CAMPAIGN_TYPE_CHANNEL_MONTH + CAMPAIGN_TYPE_MONTH + CHANNEL_MONTH
+ COST + COST_PER_FREE_TRIALS + COST_BY_FREE_TRIALS
+ LOG_COST + LOG_COST_PER_FREE_TRIALS + LOG_COST_BY_FREE_TRIALS
+ AVG_COST_MONTH + MEDIAN_COST_MONTH
+ AVG_COST_CAMPAIGN_TYPE_CHANNEL + MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL
+ AVG_FREE_TRIALS_MONTH + MEDIAN_FREE_TRIALS_MONTH
"""



# import statsmodels.api as sm
# model = sm.GLM(df['FREE_TRIALS'], sm.add_constant(X), family=sm.families.Poisson()).fit()
# print(model.summary())

model = sm.ols(formula, data=df).fit()
# Print the summary of the regression model
print(model.summary())


In [11]:
# df = pd.DataFrame({
#     'target': [1, 2, 3, 4, 5],
#     'feature1': [2, 3, 4, 5, 6],
#     'feature2': [3, 4, 5, 6, 7],
#     'feature3': [4, 5, 6, 7, 8],
#     # Add more features if needed
# })

# # Define the dependent variable
# dependent_var = 'target'

# # Generate the list of independent variables
# independent_vars = df.columns.drop(dependent_var)

# # Create the formula
# formula = f"{dependent_var} ~ " + " + ".join(independent_vars)

# # Now you can use this formula in statsmodels
# import statsmodels.formula.api as smf
# model = smf.ols(formula, data=df).fit()
# # print(model.summary())

In [12]:
# # # Define a function to plot boxplots and detect outliers based on IQR
# # def plot_outliers(column, ax):
# #     ax.boxplot(data[column], vert=False, patch_artist=True, boxprops=dict(facecolor="lightblue"))
# #     ax.set_title(f'Boxplot of {column}')
# #     ax.set_xlabel(column)

# # # Create boxplots for COST and FREE_TRIALS
# # fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# # plot_outliers('COST', axes[0])
# # plot_outliers('FREE_TRIALS', axes[1])
# # plt.tight_layout()
# # plt.show()

# # Calculate outlier thresholds using IQR
# def calculate_outliers(column):
#     q1 = data[column].quantile(0.25)
#     q3 = data[column].quantile(0.75)
#     iqr = q3 - q1
#     lower_bound = q1 - 1.5 * iqr
#     upper_bound = q3 + 1.5 * iqr
#     outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
#     return lower_bound, upper_bound, outliers

# # Remove outliers based on IQR thresholds for LOG_COST and LOG_FREE_TRIALS
# log_cost_bounds = calculate_outliers('LOG_COST')
# log_free_trials_bounds = calculate_outliers('LOG_FREE_TRIALS')

# # Filter data to exclude outliers in LOG_COST and LOG_FREE_TRIALS
# filtered_log_data = data[
#     (data['LOG_COST'] >= log_cost_bounds[0]) & (data['LOG_COST'] <= log_cost_bounds[1]) &
#     (data['LOG_FREE_TRIALS'] >= log_free_trials_bounds[0]) & (data['LOG_FREE_TRIALS'] <= log_free_trials_bounds[1])
# ]

# # Display the size of the dataset before and after outlier removal based on log variables
# log_filtered_size = filtered_log_data.shape[0]

# log_filtered_size


In [13]:
# from sklearn.preprocessing import StandardScaler

# # Select numerical columns to standardize
# numerical_features = ['LOG_COST', 'LOG_COST_SQ']
# scaler = StandardScaler()

# # Standardize numerical features
# filtered_log_data_encoded[numerical_features] = scaler.fit_transform(
#     filtered_log_data_encoded[numerical_features]
# )

# # Re-run OLS regression with standardized features
# X_standardized = filtered_log_data_encoded[['LOG_COST', 'LOG_COST_SQ'] +
#                                            [col for col in filtered_log_data_encoded.columns
#                                             if col.startswith(('ATL_OR_DR_', 'CAMPAIGN_TYPE_', 'CHANNEL_'))]]
# X_standardized = sm.add_constant(X_standardized)  # Add constant for OLS

# y_standardized = filtered_log_data_encoded['LOG_FREE_TRIALS']

# # Fit OLS regression with standardized features
# ols_standardized_model = sm.OLS(y_standardized, X_standardized).fit()

# # Display the updated summary
# ols_standardized_summary = ols_standardized_model.summary()
# ols_standardized_summary